##Przestawienie problemu. Zacznijmy od pewnego filmu. Oczywiście przedtem trzeba zaimportować odpowiednie biblioteki

In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo('Z44BZeF-ovk', width=640, height=360)

No właśnie? Czy w niedziele o godzinie 17 gorzej się gra w piłkę?

Najpierw zainstalujmy niezbędne pakiety

In [1]:
import kagglehub
import pandas as pd
from pathlib import Path, PurePath
import functions as f


x:\anaconda3\envs\analiza_danych_projekt\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Teraz zaimportujmy dane i zróbmy pierwsze modyfikacje

In [ ]:
pd.set_option('display.max_columns', 18)

path = kagglehub.dataset_download("filippomonte/premier-league-past-3-seasons")
file = PurePath(Path(path).joinpath('PremierLeagueMatches.csv'))
df_raw = pd.read_csv(file)
df_raw.columns = [x.lower() for x in df_raw.columns]
df = df_raw.dropna(subset=['result'])
#print(df.columns)
cols_to_drop = ['matchday',  'attendance', 'referee', 'stadium', 'result', '*additional stats']
df.drop(columns=cols_to_drop,inplace=True)

Index(['matchday', 'date', 'time', 'home team', 'homescore', 'homexg',
       'awayscore', 'awayxg', 'away team', 'attendance', 'referee', 'stadium',
       'result', '*additional stats'],
      dtype='object')


C:\Users\mateu\AppData\Local\Temp\ipykernel_11440\3142554772.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=cols_to_drop,inplace=True)


W następnym kroku wyciągnijmy z daty dzień tygodnia. Utworzymy 2 nowe zmienne: 
- jedna kategoryczna - nazwa dnia tygodnia
= druga czy niedziela.

Ponadto na podstawie godziny wyciągniemy czy mecz o 17. Zaokrąglijmy początek meczu do pełnej godziny (do 30 zaokrąglamy w dół od 31 w górę). Następnie stworzymy kolejną zmienną, dla wyrównania liczebności zbiorów 16-18 to jest gra o 17.

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['day_of_week'] = df['date'].dt.day_name()
df['sunday']=df['day_of_week'].apply(lambda x: 'Yes' if x=='Sunday' else 'No')
df['pure_hour'] = pd.to_datetime(df['time'], format='%H:%M')
df['pure_hour']=df['pure_hour'].apply(f.hours).dt.hour
df['seventeen']=df['pure_hour'].apply(lambda x: 'Yes' if x>=16 and x<=18 else 'No')